<a id="top"></a>
# MSTAR SAR Image Classifcation: 



## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "mstar_sar_classification_demo.ipynb", "openvino-dev-latest", ['openvino-lts'])

This sample showcases the use of the **Intel® Distribution of OpenVINO™** toolkit to optimize and deploy an internally developed ResNET18 model that classifies Synthetic Aperture Radar (SAR) images associated with 10 separate military vehicle classifiers, such as tanks and armored vehicles. The deployed model processes 3,606 Synthetic Aperture Radar (SAR) images across 10 target classifiers in order to benchmark the model’s Frames Per Second and the Seconds Per Frame across Intel’s hardware portfolio available on Dev Cloud.

# Overview of how it works
At start-up the sample application reads the command line arguments and loads a network and SAR input image to the Inference Engine (IE) plugin. A job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU and Intel® Movidius™ Neural Compute Stick 2.
After the inference is completed on all 3600+ images, the number of correct/incorrect images are stored in the /results directory.

# Demonstration objectives
 * Image as input is supported using **OpenCV**
 * Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
 * Accurate classification of Synthetic Aperture Radar images


## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    - **mstar_sar_classification_demo.ipynb** - This Jupyter* Notebook
    - **mstar_sar_classification_run_all.py** - Python* code for SAR image classification application
    - **/data/reference-sample-data/mstar-sar-classification-python/model/mstar_sar.pb** - TensorFlow frozen graph of pretrained ResNet18 model (provided)
    - **/data/reference-sample-data/mstar-sar-classification-python/TEST** - Directory containing test images for SAR model 
    - **images/HB15087.jpg** - Test image for Jupyter* Notebook demonstration 

It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>

## Set Up

### Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook. 

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [ ]:
from __future__ import print_function

import os
import sys 
import cv2

import numpy as np
import logging as log
import os.path as ops
import matplotlib.pyplot as plt

from qarpo.demoutils import *
from IPython.display import HTML
from argparse import ArgumentParser
from openvino.inference_engine import IECore
from time import time

warnings.filterwarnings('ignore',category=FutureWarning)

## Using Intel® Distribution of OpenVINO™ toolkit

First, let's try running inference on a single image to see how the Intel® Distribution of OpenVINO™ toolkit works.
We will be using Intel® Distribution of OpenVINO™ toolkit Inference Engine (IE) to classify military vehicles as seen in SAR imagery.
There are five steps involved in this task:

1. Create an Intermediate Representation (IR) Model using the Model Optimizer by Intel
2. Choose a device and create IEPlugin for the device
3. Read the Model's IR using IENetwork
4. Load the IENetwork into the Plugin
5. Run inference.

### Create Intermediate Representation of the Model using Model Optimizer

[Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) creates the Intermediate Representation of the model which is the device-agnostic, generic optimization of the model. Caffe*, TensorFlow*, MXNet*, ONNX*, and Kaldi* models are supported by Model Optimizer.

<br><div class=tip><b>Tip:</b> the '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in a command line. So the above command will work in a terminal (with '!' removed). </div>

**Run the cell below to generate an FP32 precision IR of the ResNet18 model**

In [ ]:
!mo_tf.py \
--input_model /data/reference-sample-data/mstar-sar-classification-python/model/mstar_sar.pb \
--data_type FP32 \
-o models/FP32 \
--input_shape "[1,128,128,1]"

<br><div class=note><i><b>Note: </b>the previous code cell is a single command line input, which spans 5 lines due to the backslash '\\', which is a line continuation character in Bash. </i></div>

Here, the arguments are:
* --input_model : the original model
* --data_type : Data type to use. One of {FP32, FP16, half, float}
* --input_shape: shape of data input to the model [N, H, W, C]
* -o : output directory

This script also supports `-h` that will you can get the full list of arguments that the script accepts. 

With the `-o` option set as above, this command will write the output to the directory `model/FP32`

There are two files produced:
```
models/FP32/mstar_sar.xml
models/FP32/mstar_sar.bin
```
These will be used later in the exercise.

**We will also build an FP16 precision IR for the model using the same process.**

This will produce two files as well:
```
models/FP16/mstar_sar.xml
models/FP16/mstar_sar.bin
```

In [ ]:
!mo_tf.py \
--input_model /data/reference-sample-data/mstar-sar-classification-python/model/mstar_sar.pb \
--data_type FP16 \
-o models/FP16 \
--input_shape "[1,128,128,1]"

## Run Inference Example
Initially we will build an example of running inference on the an image of a **ZSU-23-4 tank**. The image on the **left** shows the tank from the side with a standard RGB camera, while the SAR image looks significantly different as shown on the **right** due to being taken from overhead and being captured using SAR image sensors.   

<img src="images/640px-ZSU-23-4-Camp-Pendleton.jpg" height="213" width="320" align="left" title="By Sgt. Ryan Ward, U.S. Marine Corps, Public Domain, https://commons.wikimedia.org/w/index.php?curid=478813" />



<img src="images/HB15087.jpg" align="center"/>

### Define variables

In [ ]:
model_xml='models/FP32/mstar_sar.xml'
device_arg='CPU'
input_arg=['images/HB15087.jpg']
class_labels=['ZSU_23_4']
iterations=10
perf_counts=False
labels = np.array(['2S1', 'BMP2', 'BRDM_2', 'BTR70', 'BTR_60', 'D7', 'T62', 'T72', 'ZIL131', 'ZSU_23_4'])

log.basicConfig(format="[ %(levelname)s ] %(message)s", level=log.INFO, stream=sys.stdout)

### Inference Engine initialization  and load extensions library
We initialize the Inference Engine by calling the class **`IECore()`**. For more details on **`IECore`** see the <a href=https://docs.openvinotoolkit.org/latest/ie_python_api/classie__api_1_1IECore.html>IECore Documentation</a>.

In [ ]:
ie = IECore()

### Read IR
We can use the **`IECore`** function **`read_network`** to import the optimized network IR.

In [ ]:
model_bin = os.path.splitext(model_xml)[0] + ".bin"
net = ie.read_network(model=model_xml, weights=model_bin)

### Preparing input blobs

In [ ]:
input_blob = next(iter(net.input_info))
out_blob = next(iter(net.outputs))
net.batch_size = len(input_arg)

### Read and pre-process input images
First let's load the image using OpenCV.
We will also have to do some shape manipulation to convert the image to a format that is compatible with our network.

In [ ]:
n, c, h, w = net.input_info[input_blob].input_data.shape
images = np.ndarray(shape=(n, c, h, w))
for i in range(n):
    image = cv2.imread(input_arg[i], 0) # Read image as greyscale
    if image.shape[:-1] != (h, w):
        log.warning("Image {} is resized from {} to {}".format(input_arg[i], image.shape, (h, w)))
        image = cv2.resize(image, (w, h))
        
    # Normalize to keep data between 0 - 1
    image = (np.array(image) - 0) / 255.0

    # Change data layout from HWC to CHW
    image = image.reshape((1, 1, h, w))    
    images[i] = image
    
log.info("Batch size is {}".format(n))

### Loading model to the plugin
Once we have the Inference Engine and the network, we can load the network into the Inference Engine using **`ie.load_network`**.

In [ ]:
exec_net = ie.load_network(network=net, device_name=device_arg)

### Run Inference
We can now run inference on the object  **`exec_net`** using the **`infer`** function call.

In [ ]:
infer_time = []
for i in range(iterations):
    t0 = time()
    result = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time()-t0)*1000)

result = result[out_blob]
    
log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))

### Processing output blob
The network outputs a tensor of dimension 10 (number of classes), these values represent the probability that the image is a particular class. To make the final classification we find the class with the highest probability. 

In [ ]:
# Access the results and get the index of the highest confidence score

# Predicted class index.
for index, res in enumerate(result):
    
    class_num = np.argmax(res)
    if labels[class_num] == class_labels[index]:
        print ("Result correctly classified as: {}".format(labels[class_num]))
    else:
        print ("Result incorrectly classified as: {}".format(labels[class_num]))


## Job Submission

The inference code is already implemented in 
<a href="mstar_sar_classification_run_all.py">mstar_sar_classification_run_all.py</a>.

The Python code takes in command line arguments for images, model etc.

**Command line arguments options and how they are interpreted in the application source code**

```
python3 mstar_sar_classification_run_all.py  -m ${MODELPATH} \
                                             -i ${INPUT_ADDR} \
                                             -o ${OUTPUT_DIR} \
                                             -d ${DEVICE} \
                                             -pc
```

**The description of the arguments used in the argument parser is the command line executable equivalent.**
* -m Location of the model's IR file (.xml + .bin) which has been converted using the **model optimizer**.
   There is automated support built in this argument to support both FP32 and FP16 models targeting different hardware
* -i Path of the input images 
* -o Location where the output file with inference needs to be stored. (results/)
* -d Type of Hardware Acceleration (CPU, GPU, MYRIAD)
* -l Absolute path to extension library file to load to a plugin. (Optional)
* -pc Report individual model layer performance counts will be reported to log file in the results/ directory (or as specified with -o argument) 

### Creating the job file

We will run inference on several different edge compute nodes present in the Intel® DevCloud for the Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that will be executed directly on the edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
mstar_sar_job.sh <output_directory> <device> <fp_precision> <input_file> 
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*device*> - Hardware device to use (e.g. CPU, GPU, etc.)
- <*fp_precision*> - Which floating point precision inference model to use (FP32 or FP16)
- <*input_file*> - Path to input image file(s)

Run the following cell to create the `mstar_sar_job.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `mstar_sar_job.sh`.

In [ ]:
%%writefile mstar_sar_job.sh

# MSTAR SAR job script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
OUTPUT_DIR=$1
DEVICE=$2
FP_MODEL=$3
INPUT_ADDR=$4

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
mkdir -p $1
SAMPLEPATH=$PBS_O_WORKDIR
python3 mstar_sar_classification_run_all.py    -m models/$3/mstar_sar.xml  \
                                               -i $4 \
                                               -o $1 \
                                               -d $2 \
                                               -pc


### Understand how jobs are submitted into the queue

Now that we have the job script, we can submit the jobs to edge compute nodes. In the IoT DevCloud, you can do this using the `qsub` command.
We can submit `mstar_sar_job.sh` to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets us name the job so that it is easier to distinguish between them.

The `-F` flag is used to pass in arguments to the job script.
The [mstar_sar_job.sh](mstar_sar_job.sh) takes in 4 arguments:
1. the path to the directory for the performance stats
2. targeted device (e.g. CPU, GPU, MYRIAD)
3. the floating precision to use for inference
4. the path to the input images

The job scheduler will use the contents of `-F` flag as the argument to the job script.

If you are curious to see the available types of nodes on the IoT DevCloud, run the following optional cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

Here, the properties describe the node, and number on the left is the number of available nodes of that architecture.

### Job queue submission

Each of the 5 cells below will submit a job to different edge compute nodes.
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

<br><div class=note><i><b>Note: </b>You may submit all jobs at once or one at a time. After submission, they will go into a queue and run as soon as the requested compute resources become available.</i></div> 
 

<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>

In [ ]:
os.environ["IMAGES"] = "/data/reference-sample-data/mstar-sar-classification-python/TEST"

#### Submitting to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/content/www/us/en/ark/products/97121/intel-core-i5-7500t-processor-6m-cache-up-to-3-30-ghz.html">Intel® Core™ i5-7500T processor</a>. The inference workload will run the CPU.

In [ ]:
#Submit job to the queue
job_id_core_kaby = !qsub mstar_sar_job.sh -l nodes=1:idc006kbl -F "results/ CPU FP32 $IMAGES" -N mstar_sar_core_kaby
print(job_id_core_kaby[0])
jobid_core_kaby = job_id_core_kaby[0].split('.')[0]
#Progress indicators
if job_id_core_kaby:
    progressIndicator('results/', 'i_progress_'+job_id_core_kaby[0]+'.txt', "Inference", 0, 100)
else:
    print("Error in job submission.")

##### Submit to an edge compute node with Intel® Xeon® Gold 6258R CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Gold 6258R Processor](https://ark.intel.com/content/www/us/en/ark/products/199350/intel-xeon-gold-6258r-processor-38-5m-cache-2-70-ghz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_cascade_lake = !qsub mstar_sar_job.sh -l nodes=1:idc018 -F "results/ CPU FP32 $IMAGES" -N mstar_sar_xeon_cascade_lake 
print(job_id_xeon_cascade_lake[0]) 
jobid_xeon_cascade_lake = job_id_xeon_cascade_lake[0].split('.')[0]
#Progress indicators
if job_id_xeon_cascade_lake:
    progressIndicator('results/', 'i_progress_'+job_id_xeon_cascade_lake[0]+'.txt', "Inference", 0, 100)
else:
    print("Error in job submission.")

##### Submit to an edge compute node with Intel® Xeon® E3-1268L v5 CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_skylake = !qsub mstar_sar_job.sh -l nodes=1:idc007xv5 -F "results/ CPU FP32 $IMAGES" -N mstar_sar_xeon_skylake 
print(job_id_xeon_skylake[0]) 
jobid_xeon_skylake = job_id_xeon_skylake[0].split('.')[0]
#Progress indicators
if job_id_xeon_skylake:
    progressIndicator('results/', 'i_progress_'+job_id_xeon_skylake[0]+'.txt', "Inference", 0, 100)
else:
    print("Error in job submission.")

#### Submitting to an edge compute node with Intel® Core i7 CPU and using the onboard Intel® GPU (UHD-620)
In the cell below, we submit a job to an <a 
    href="https://www.aaeon.com/en/p/iot-gateway-node-systems-upx-edge">UPX-Edge</a> edge node with an <a href="https://ark.intel.com/content/www/us/en/ark/products/193554/intel-core-i7-8665ue-processor-8m-cache-up-to-4-40-ghz.html">Intel® Core i7-8665UE</a>. The inference workload will run on the Intel® UHD Graphics 620 card integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_corei7_gpu = !qsub mstar_sar_job.sh -l nodes=1:idc014upxa10fx1 -F "results/ GPU FP32 $IMAGES" -N mstar_sar_corei7_gpu
print(job_id_corei7_gpu[0])
jobid_corei7_gpu = job_id_corei7_gpu[0].split('.')[0]
#Progress indicators
if job_id_corei7_gpu:
    progressIndicator('results/', 'i_progress_'+job_id_corei7_gpu[0]+'.txt', "Inference", 0, 100)
else:
    print("Error in job submission.")

#### Submitting to an edge compute node with Intel® Core i5 CPU and using the onboard Intel® GPU (HD-630)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a href="https://ark.intel.com/content/www/us/en/ark/products/97121/intel-core-i5-7500t-processor-6m-cache-up-to-3-30-ghz.html">Intel® Core i5-7500T</a>. The inference workload will run on the Intel® HD Graphics 630 card integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_corei5gpu = !qsub mstar_sar_job.sh -l nodes=1:idc006kbl -F "results/ GPU FP32 $IMAGES" -N mstar_sar_corei5gpu
print(job_id_corei5gpu[0])
jobid_corei5gpu = job_id_corei5gpu[0].split('.')[0]
#Progress indicators
if job_id_corei5gpu:
    progressIndicator('results/', 'i_progress_'+job_id_corei5gpu[0]+'.txt', "Inference", 0, 100)
else:
    print("Error in job submission.")

### Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up. 

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job(s)).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

<br><div class=danger><b>Wait!: </b>Please wait for the inference jobs to complete before proceeding to the next step to view results.</div>


## View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`mstar_sar_{type}.o{JobID}`

`mstar_sar_{type}.e{JobID}`

(here, mstar_sar_{type} corresponds to the `-N` option of qsub).


## Assess Performance

The running time of each inference task is recorded in `results/stats_job_id_{ARCH}.txt`, where the subdirectory name corresponds to the architecture of the target edge compute node. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core_kaby', 'Intel Core\ni5-7500T\nCPU'),
             ('xeon_cascade_lake', 'Intel Xeon\nGold\n 6258R\nCPU'),
             ('xeon_skylake', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('corei7_gpu', ' Intel Core\ni7-8665UE\nGPU'),
             ('corei5gpu', 'Intel\nCore\ni5-7500T\nGPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/stats_'+vars()['job_id_'+arch][0]+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time per Image, seconds', 'Inference Engine Processing Time per Image', 'time' )

summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )

## Telemetry Dashboard

Once your submitted jobs are completed, run the cells below to generate links to view telemetry dashboards containing performance metrics for your model and target architecture.

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core™ i5-7500T</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + jobid_core_kaby

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® Gold 6258R CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + jobid_xeon_cascade_lake

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Xeon® E3-1268L CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + jobid_xeon_skylake

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Core i7-8665UE CPU and using the onboard Intel® GPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + jobid_corei7_gpu

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view metering dashboard of the last job ran on Intel® Core i5-7500T CPU and using the onboard Intel® GPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + jobid_corei5gpu

html = HTML(link_t.format(href=result_file))

display(html)

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge

## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>